In [1]:
#Read Data From CSV file
import pandas as pd
df=pd.read_csv('NEWDATATABLE.csv', sep=',',header=None)
ratings = df.values

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
TopN_recommend = ratings.copy()
for indexI in range(24983):
    for indexJ in range(100):
        if TopN_recommend[indexI][indexJ] == 99:
            TopN_recommend[indexI][indexJ] = 0
        else:
            TopN_recommend[indexI][indexJ] = 1

In [4]:
def train_test_generation(ratings):
    #Cretae train and test matrix
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in xrange(ratings.shape[0]):
        #Randomly pick 10 ratings from each user to be in test case (include unknown ratings)
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size = 20, replace = False)
        #Set chosen be zero in training
        train [user, test_ratings] = 0.
        #Assign chosen rating to test case
        test[user, test_ratings] = ratings[user, test_ratings]
    
    #Make sure train and test are disjoint 
    assert(np.all((train * test) == 0))
    return train, test

In [5]:
#Call the function and then assign train and test
train, test = train_test_generation(TopN_recommend)
trainMAE = train.copy()

In [6]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(train, k=10)
sigma = np.diag(sigma)
#predicted_TopN_recommend is used for reducing to lower dimension
predicted_TopN_recommend = np.dot(np.dot(U, sigma), Vt)

In [8]:
#Full dimension 
predicted_TopN_recommend = train

In [9]:
#Fcn to compute user or item similarity
from sklearn.metrics.pairwise import cosine_similarity

def similarity(ratings, kind='user'):
    if kind == 'user':
        sim = cosine_similarity(ratings)
        assert(sim.shape[0] == ratings.shape[0])
    elif kind == 'item':
        sim = cosine_similarity(ratings.T)
        assert(sim.shape[0] == ratings.shape[1])
    sim[np.isnan(sim)] = 0
    np.fill_diagonal(sim,0)
    return sim

In [10]:
#Compute user similarity
user_similarity_TopN_recommend = similarity(predicted_TopN_recommend, kind='user')
#Copy them for finding index in KNN
user_similarity_fcn_TopN_recommend = user_similarity_TopN_recommend.copy()

In [ ]:
#Find KNN thu sklearn lib
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=5).fit(user_similarity_fcn_TopN_recommend)
KNN_result = nbrs.kneighbors_graph(user_similarity_fcn_TopN_recommend).toarray()

In [ ]:
from collections import defaultdict
KNN_index_result = defaultdict(list)

for indexI in range(24983):
    temp_KNN_index = []
    #Conversion for call sklearn KNN lib
    temp_KNN_index = list(KNN_result[indexI].nonzero()[0])
    KNN_index_result[indexI].append(temp_KNN_index)

In [52]:
#Recommend Top N items and save their item index into NumberMax List
L = len(KNN_result)
NumberMax = []
for indexI in range(L):
    temp_total= np.zeros((100,))
    for indexJ in range(k):
        temp_total = np.add(temp_total, TopN_recommend[KNN_index_result[indexI][0][indexJ]])
        
    Maxtemp = max(temp_total)
    NumberMax.append(np.where(temp_total == Maxtemp)[0])

In [124]:
#Compute recall and precise for each user
def recall_precise(index_matrix, test_index):
    recall_result = []
    precise_result = []
    
    for indexI in range(len(index_matrix)):
        count = 0
        index_matrix_conversion = list(index_matrix[indexI])
        for item in index_matrix_conversion:
            if test_index[indexI][item] == 1:
                count = count + 1
        recall_temp = float(count) / 20
        precise_temp = float(count) / len(index_matrix[indexI])
        recall_result.append(recall_temp)
        precise_result.append(precise_temp)
    return recall_result, precise_result

In [131]:
#Compute average user metric and then find final mertic 
def F1_metric(recall, precise):
    F1_result = [] 
    for indexI in range(len(recall)):
        F1_temp = 2 * recall[indexI] * precise[indexI] / float(recall[indexI] + precise[indexI])
    F1_result.append(F1_temp)
    F1_AVG = sum(F1_result) / float(len(F1_result))
    return F1_AVG

In [132]:
#Call two fcns to achieve to find
recall_each, precise_each = recall_precise(NumberMax, test)
F1_AVG_result = F1_metric(recall_each, precise_each)